<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220106_sm_lgbm_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification Knowcode using KoBERT Model

할일 : 텍스트로 변형 가능한 데이터 찾기

예 : 최종학력, 전공, 성별, 등

In [1]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-z8za1d6_/kobert-tokenizer_7763d37fbe4a43c9ae73da969041e60a
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-z8za1d6_/kobert-tokenizer_7763d37fbe4a43c9ae73da969041e60a
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-0b1xeo8c
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-0b1xeo8c


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
import warnings

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
warnings.filterwarnings(action='ignore') 
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]

In [4]:
train1

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,...,bq18_4,bq18_5,bq18_6,bq18_7,bq19,bq19_1,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,...,3,2,3,3,2,생산설비의 자동화로,2,3,5,10.0,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,없다,없다,건설현장 노무직,없다,4,1,52,2,실업,1,1,1,4000,,2200,825101
1,1,4,5,4,5,3,4,3,4,3,4,1,,1,,1,,1,,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,,2,...,4,2,2,4,2,건설 수주가 없어서,3,3,6,50.0,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,"매타기, 드라이버, 가위, CAD",,,,2,1,38,4,건축공학,1,1,1,,,2400,140204
2,2,3,4,3,4,3,4,5,6,4,5,1,,1,,3,4,3,4,3,4,1,,3,4,3,4,1,,3,4,3,4,3,4,3,4,3,4,3,...,4,2,2,4,2,신축 건설경기가 좋지 않아서,3,2,5,40.0,2,2,2,2,2,2,2,2,4,4,4,2,2,없다,"건축설계표, 오토캐드, 파이프, 보일러",,,,2,1,50,4,건축공학,1,1,1,4000,,2400,140204
3,3,3,3,3,3,3,5,4,5,4,6,3,5,3,4,4,5,5,5,4,5,3,4,4,5,3,5,3,4,4,4,3,4,2,2,4,4,4,2,4,...,3,2,2,3,1,"업무량 감소, 자동화로 인해서",4,2,6,30.0,2,1,1,2,1,1,1,1,3,4,4,4,4,지적기사,"토탈스테이션, SZP",무,지적공무원,,4,1,42,4,환경학과,1,1,1,7000,,3500,140601
4,4,4,5,3,4,3,4,4,5,3,4,1,,1,,1,,1,,3,4,2,2,3,4,3,4,1,,3,4,3,4,3,4,3,4,1,,2,...,4,2,2,4,2,부동산 거품 빠지면서 건축 경기도 덩달아 불황,3,2,3,40.0,2,2,2,2,2,2,2,2,4,3,4,3,3,없다,"캐드, 엑셀, 건축도면",,,,2,1,51,4,건축공학,1,1,1,4000,,2500,140204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,5,2,4,3,3,2,2,2,3,2,3,3,3,2,3,2,3,3,4,4,5,4,5,3,4,2,3,3,4,2,2,2,2,2,2,3,5,1,...,4,3,2,3,3,출판업무 특성상 기획업무는 변함없음,2,3,1,10.0,2,1,2,2,2,2,2,2,4,4,4,4,3,없다,"한글, 편집프로그램, 포토샵, 일러스트",없다,마케팅업무,없다,2,1,50,4,산업디자인,1,1,1,5200,,1800,411301
9482,9482,5,5,5,5,5,5,3,4,4,5,5,5,4,4,5,5,4,5,4,5,4,5,5,5,4,5,4,5,5,4,2,2,3,4,4,5,5,5,3,...,4,1,2,4,4,항공우주분야는 국가 경쟁력을 위해 육성해야 되는 분야이므로,3,3,5,20.0,2,2,1,2,2,2,2,2,4,4,3,4,3,없다,"AUTO CAD , CATIA, 전문 특수정비도구(툴)",없다,없다,없다,2,1,37,4,우주항공공학,1,1,1,4000,,3000,151105
9483,9483,3,3,4,6,3,3,4,5,4,5,3,4,3,4,1,,2,2,3,3,3,4,3,4,3,4,3,4,3,3,5,7,5,7,4,6,1,,4,...,2,4,3,3,3,건설경기는 꾸준해서,2,3,1,0.0,2,2,2,2,2,2,2,2,3,3,4,3,1,건설 노가대,"볼트, 리벳",없다,없다,없다,2,1,32,2,기계,1,4,2,2700,,1500,701101
9484,9484,3,5,3,5,4,5,3,4,3,5,4,6,4,5,4,5,4,6,4,6,3,4,3,5,3,4,2,3,4,6,3,3,3,4,3,3,4,5,4,...,3,3,2,3,3,공무원의 인원은 크게 변동이 없어서,2,2,5,20.0,2,1,1,2,1,2,1,1,3,4,4,3,2,없다,컴퓨터,없다,모름,,2,2,40,4,문헌정보학,1,1,1,6800,,2500,25402


In [5]:
def tool_to_feature(text:str):
  result = []
  for i in text.split(','):
    result.append(i)
  return result

train1['bq31'] = train1['bq31'].map(tool_to_feature)

In [6]:
def list_truncate(tool:list):
  if len(tool) > 3:
    return tool[:3]
  elif len(tool) == 3:
    return tool
  elif len(tool) == 2:
    return tool+['0']
  elif len(tool) == 1:
    return tool+['0', '0']
  elif len(tool) == 0:
    return ['0', '0', '0']

train1['bq31'] = train1['bq31'].map(list_truncate)

In [7]:
def list_to_feature1(tool:list):
  return tool[0]

def list_to_feature2(tool:list):
  return tool[1]

def list_to_feature3(tool:list):
  return tool[2]


feature1 = train1['bq31'].map(list_to_feature1)
feature2 = train1['bq31'].map(list_to_feature2)
feature3 = train1['bq31'].map(list_to_feature3)

In [8]:
train1['feature1'] = feature1
train1['feature2'] = feature2
train1['feature3'] = feature3

In [9]:
train1[['feature1', 'feature2', 'feature3']]

,feature1,feature2,feature3
0,없다,0,0
1,매타기,드라이버,가위
2,건축설계표,오토캐드,파이프
3,토탈스테이션,SZP,0
4,캐드,엑셀,건축도면
...,...,...,...
9481,한글,편집프로그램,포토샵
9482,AUTO CAD,CATIA,전문 특수정비도구(툴)
9483,볼트,리벳,0
9484,컴퓨터,0,0


In [10]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
cols = ['feature1', 'feature2', 'feature3']
temp = train1[cols[0]].append(train1[cols[1]])
temp = temp.append(train1[cols[2]])
encoder = encoder.fit(temp)
for col in cols:
  train1[col] = train1[col].map(str)
  train1[col] = encoder.transform(train1[col])

In [11]:
train1.drop('bq31', axis=1)

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,...,bq18_6,bq18_7,bq19,bq19_1,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode,feature1,feature2,feature3
0,0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,...,3,3,2,생산설비의 자동화로,2,3,5,10.0,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,없다,건설현장 노무직,없다,4,1,52,2,실업,1,1,1,4000,,2200,825101,5236,2846,2846
1,1,4,5,4,5,3,4,3,4,3,4,1,,1,,1,,1,,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,,2,...,2,4,2,건설 수주가 없어서,3,3,6,50.0,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,,,,2,1,38,4,건축공학,1,1,1,,,2400,140204,4208,817,416
2,2,3,4,3,4,3,4,5,6,4,5,1,,1,,3,4,3,4,3,4,1,,3,4,3,4,1,,3,4,3,4,3,4,3,4,3,4,3,...,2,4,2,신축 건설경기가 좋지 않아서,3,2,5,40.0,2,2,2,2,2,2,2,2,4,4,4,2,2,없다,,,,2,1,50,4,건축공학,1,1,1,4000,,2400,140204,3494,1799,2562
3,3,3,3,3,3,3,5,4,5,4,6,3,5,3,4,4,5,5,5,4,5,3,4,4,5,3,5,3,4,4,4,3,4,2,2,4,4,4,2,4,...,2,3,1,"업무량 감소, 자동화로 인해서",4,2,6,30.0,2,1,1,2,1,1,1,1,3,4,4,4,4,지적기사,무,지적공무원,,4,1,42,4,환경학과,1,1,1,7000,,3500,140601,6448,308,2846
4,4,4,5,3,4,3,4,4,5,3,4,1,,1,,1,,1,,3,4,2,2,3,4,3,4,1,,3,4,3,4,3,4,3,4,1,,2,...,2,4,2,부동산 거품 빠지면서 건축 경기도 덩달아 불황,3,2,3,40.0,2,2,2,2,2,2,2,2,4,3,4,3,3,없다,,,,2,1,51,4,건축공학,1,1,1,4000,,2500,140204,6260,1714,473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,5,2,4,3,3,2,2,2,3,2,3,3,3,2,3,2,3,3,4,4,5,4,5,3,4,2,3,3,4,2,2,2,2,2,2,3,5,1,...,2,3,3,출판업무 특성상 기획업무는 변함없음,2,3,1,10.0,2,1,2,2,2,2,2,2,4,4,4,4,3,없다,없다,마케팅업무,없다,2,1,50,4,산업디자인,1,1,1,5200,,1800,411301,6709,2612,2636
9482,9482,5,5,5,5,5,5,3,4,4,5,5,5,4,4,5,5,4,5,4,5,4,5,5,5,4,5,4,5,5,4,2,2,3,4,4,5,5,5,3,...,2,4,4,항공우주분야는 국가 경쟁력을 위해 육성해야 되는 분야이므로,3,3,5,20.0,2,2,1,2,2,2,2,2,4,4,3,4,3,없다,없다,없다,없다,2,1,37,4,우주항공공학,1,1,1,4000,,3000,151105,2899,69,2097
9483,9483,3,3,4,6,3,3,4,5,4,5,3,4,3,4,1,,2,2,3,3,3,4,3,4,3,4,3,4,3,3,5,7,5,7,4,6,1,,4,...,3,3,3,건설경기는 꾸준해서,2,3,1,0.0,2,2,2,2,2,2,2,2,3,3,4,3,1,건설 노가대,없다,없다,없다,2,1,32,2,기계,1,4,2,2700,,1500,701101,4547,903,2846
9484,9484,3,5,3,5,4,5,3,4,3,5,4,6,4,5,4,5,4,6,4,6,3,4,3,5,3,4,2,3,4,6,3,3,3,4,3,3,4,5,4,...,2,3,3,공무원의 인원은 크게 변동이 없어서,2,2,5,20.0,2,1,1,2,1,2,1,1,3,4,4,3,2,없다,없다,모름,,2,2,40,4,문헌정보학,1,1,1,6800,,2500,25402,6286,2846,2846


In [12]:
category_feature = []

for col in train1.columns:
    if col == 'ID':
        continue
    try:
        train1[col] = train1[col].map(int)
    except:
        category_feature.append(col)
        encoder = LabelEncoder()
        train1[col] = train1[col].map(str)
        train1[col] = encoder.fit_transform(train1[col])

In [13]:
category_feature = category_feature+['feature1', 'feature2', 'feature3']

In [14]:
category_feature

['aq1_2',
 'aq2_2',
 'aq3_2',
 'aq4_2',
 'aq5_2',
 'aq6_2',
 'aq7_2',
 'aq8_2',
 'aq9_2',
 'aq10_2',
 'aq11_2',
 'aq12_2',
 'aq13_2',
 'aq14_2',
 'aq15_2',
 'aq16_2',
 'aq17_2',
 'aq18_2',
 'aq19_2',
 'aq20_2',
 'aq21_2',
 'aq22_2',
 'aq23_2',
 'aq24_2',
 'aq25_2',
 'aq26_2',
 'aq27_2',
 'aq28_2',
 'aq29_2',
 'aq30_2',
 'aq31_2',
 'aq32_2',
 'aq33_2',
 'aq34_2',
 'aq35_2',
 'aq36_2',
 'aq37_2',
 'aq38_2',
 'aq39_2',
 'aq40_2',
 'aq41_2',
 'bq4_1a',
 'bq4_1b',
 'bq4_1c',
 'bq5_1',
 'bq5_2',
 'bq12_2',
 'bq12_3',
 'bq12_4',
 'bq19_1',
 'bq30',
 'bq31',
 'bq32',
 'bq33',
 'bq34',
 'bq38_1',
 'bq40',
 'bq41_1',
 'bq41_2',
 'bq41_3',
 'feature1',
 'feature2',
 'feature3']

In [15]:
# train1[category_feature] = train1[category_feature].astype('category')

In [16]:
feature = train1.columns.drop(['knowcode', 'idx'])
X = train1[feature]
y = train1['knowcode']

In [26]:
len(y.unique())

538

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, shuffle=True)

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
model = RandomForestClassifier()

In [37]:
model.fit(X_train, y_train)

RandomForestClassifier()

In [40]:
print(model.score(X_test, y_test))

0.5147523709167545


In [41]:
y_pred=model.predict(X_test)

In [18]:
import lightgbm as lgb

In [29]:
clf = lgb.LGBMClassifier(silent=False)
clf.fit(X_train, y_train, verbose=2)

LGBMClassifier(silent=False)

In [30]:
y_pred=clf.predict(X_test)

In [32]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print(accuracy)

0.04162276080084299


In [33]:
y_pred

array([ 13902, 811401, 812901, ..., 621402, 872101,  29501])

In [34]:
y_test

3646     27201
1349    841101
8633    812901
1296    307101
7288    155305
         ...  
8455    901501
6263    416701
6941     26201
9204     14103
9158    151108
Name: knowcode, Length: 1898, dtype: int64